In [1]:
import pandas as pd
import features_processing_utils as fpu

In [2]:
politifact_fake = pd.read_csv("FakeNewsNet/politifact_fake.csv")
politifact_fake.head()

,id,news_url,title,tweet_ids
0,politifact15014,speedtalk.com/forum/viewtopic.php?t=51650,BREAKING: First NFL Team Declares Bankruptcy O...,937349434668498944\t937379378006282240\t937380...
1,politifact15156,politics2020.info/index.php/2018/03/13/court-o...,Court Orders Obama To Pay $400 Million In Rest...,972666281441878016\t972678396575559680\t972827...
2,politifact14745,www.nscdscamps.org/blog/category/parenting/467...,UPDATE: Second Roy Moore Accuser Works For Mic...,929405740732870656\t929439450400264192\t929439...
3,politifact14355,https://howafrica.com/oscar-pistorius-attempts...,Oscar Pistorius Attempts To Commit Suicide,886941526458347521\t887011300278194176\t887023...
4,politifact15371,http://washingtonsources.org/trump-votes-for-d...,Trump Votes For Death Penalty For Being Gay,915205698212040704\t915242076681506816\t915249...


In [3]:
politifact_fake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         432 non-null    object
 1   news_url   428 non-null    object
 2   title      432 non-null    object
 3   tweet_ids  392 non-null    object
dtypes: object(4)
memory usage: 13.6+ KB


In [4]:
politifact_real = pd.read_csv("FakeNewsNet/politifact_real.csv")
politifact_real.head()

,id,news_url,title,tweet_ids
0,politifact14984,http://www.nfib-sbet.org/,National Federation of Independent Business,967132259869487105\t967164368768196609\t967215...
1,politifact12944,http://www.cq.com/doc/newsmakertranscripts-494...,comments in Fayetteville NC,942953459\t8980098198\t16253717352\t1668513250...
2,politifact333,https://web.archive.org/web/20080204072132/htt...,"Romney makes pitch, hoping to close deal : Ele...",NaN
3,politifact4358,https://web.archive.org/web/20110811143753/htt...,Democratic Leaders Say House Democrats Are Uni...,NaN
4,politifact779,https://web.archive.org/web/20070820164107/htt...,"Budget of the United States Government, FY 2008",89804710374154240\t91270460595109888\t96039619...


In [5]:
politifact_real.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 624 entries, 0 to 623
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         624 non-null    object
 1   news_url   567 non-null    object
 2   title      624 non-null    object
 3   tweet_ids  409 non-null    object
dtypes: object(4)
memory usage: 19.6+ KB


In [6]:
politifact_fake_posts = pd.read_csv("data/politifact_fake_posts.csv")

In [7]:
# politifact_fake_posts = fpu.create_dataset(politifact_fake)
# politifact_fake_posts.to_csv("data/politifact_fake_posts.csv", index=False)

In [8]:
# politifact_fake_features = fpu.get_features(politifact_fake_posts, 0)
# politifact_fake_features.head()

In [9]:
# politifact_fake_features.info()

In [10]:
# politifact_real_posts = pd.read_csv("data/politifact_real_posts.csv")

In [11]:
# politifact_real_posts = fpu.create_dataset(politifact_real)
# politifact_real_posts.to_csv("data/politifact_real_posts.csv", index=False)

In [12]:
# politifact_real_features = fpu.get_features(politifact_real_posts, 1)
# politifact_real_features.head()

In [13]:
# politifact_real_features.info()

In [14]:
# politifact_features = pd.concat([politifact_fake_features, politifact_real_features])
# politifact_features.info()

In [15]:
# politifact_features.to_csv("data/politifact_features.csv", index=False)

In [16]:
politifact_features = pd.read_csv("data/politifact_features.csv")

In [17]:
gossipcop_fake = pd.read_csv("FakeNewsNet/gossipcop_fake.csv")

In [18]:
gossipcop_fake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5323 entries, 0 to 5322
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         5323 non-null   object
 1   news_url   5067 non-null   object
 2   title      5323 non-null   object
 3   tweet_ids  5135 non-null   object
dtypes: object(4)
memory usage: 166.5+ KB


In [ ]:
# gossipcop_fake_posts = fpu.create_dataset(gossipcop_fake[:500])
# gossipcop_fake_posts.to_csv("data/gossipcop_fake_posts_1.csv", index=False)
# gossipcop_fake_features = fpu.get_features(gossipcop_fake_posts, 0)
# gossipcop_fake_features.to_csv("data/gossipcop_fake_features_1.csv", index=False)
# gossipcop_fake_features.head()

In [ ]:
# gossipcop_fake_posts = fpu.create_dataset(gossipcop_fake[500:1000])
# gossipcop_fake_posts.to_csv("data/gossipcop_fake_posts_1_bis.csv", index=False)
# gossipcop_fake_features = fpu.get_features(gossipcop_fake_posts, 0)
# gossipcop_fake_features.to_csv("data/gossipcop_fake_features_1_bis.csv", index=False)
# gossipcop_fake_features.head()

In [21]:
# gossipcop_fake_features.info()

In [22]:
gossipcop_fake_posts = fpu.create_dataset(gossipcop_fake[1000:2000])
gossipcop_fake_posts.to_csv("data/gossipcop_fake_posts_2.csv", index=False)
gossipcop_fake_features = fpu.get_features(gossipcop_fake_posts, 0)
gossipcop_fake_features.to_csv("data/gossipcop_fake_features_2.csv", index=False)
gossipcop_fake_features.info()

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 6051/6051 [52:16<00:00,  1.93it/s]   


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88 entries, 0 to 87
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   news_id                    88 non-null     object 
 1   average followers          88 non-null     float64
 2   average follows            88 non-null     float64
 3   repost total               88 non-null     int32  
 4   post total                 88 non-null     int32  
 5   repost percentage          88 non-null     float64
 6   average repost             88 non-null     float64
 7   average favorite           88 non-null     float64
 8   label                      88 non-null     int64  
 9   news lifetime              88 non-null     int32  
 10  nb users 10 hours          88 non-null     int32  
 11  average time difference    88 non-null     float64
 12  retweet percentage 1 hour  88 non-null     float64
dtypes: float64(7), int32(4), int64(1), object(1)
memory 

In [23]:
gossipcop_fake_posts = fpu.create_dataset(gossipcop_fake[2000:3000])
gossipcop_fake_posts.to_csv("data/gossipcop_fake_posts_3.csv", index=False)
gossipcop_fake_features = fpu.get_features(gossipcop_fake_posts, 0)
gossipcop_fake_features.to_csv("data/gossipcop_fake_features_3.csv", index=False)
gossipcop_fake_features.info()

 11%|█         | 111/1000 [10:02<1:20:25,  5.43s/it]


KeyboardInterrupt: 

In [ ]:
gossipcop_fake_posts = fpu.create_dataset(gossipcop_fake[3000:4000])
gossipcop_fake_posts.to_csv("data/gossipcop_fake_posts_4.csv", index=False)
gossipcop_fake_features = fpu.get_features(gossipcop_fake_posts, 0)
gossipcop_fake_features.to_csv("data/gossipcop_fake_features_4.csv", index=False)
gossipcop_fake_features.info()

In [ ]:
gossipcop_fake_posts = fpu.create_dataset(gossipcop_fake[4000:])
gossipcop_fake_posts.to_csv("data/gossipcop_fake_posts_5.csv", index=False)
gossipcop_fake_features = fpu.get_features(gossipcop_fake_posts, 0)
gossipcop_fake_features.to_csv("data/gossipcop_fake_features_5.csv", index=False)
gossipcop_fake_features.info()

In [ ]:
gossipcop_real = pd.read_csv("FakeNewsNet/gossipcop_real.csv")

In [ ]:
fpu.complete_processing(gossipcop_real, 1, "gossipcop_real_posts_1.csv", "gossipcop_real_features_1.csv", 0, 1000)

In [ ]:
fpu.complete_processing(gossipcop_real, 1, "gossipcop_real_posts_2.csv", "gossipcop_real_features_2.csv", 1000, 2000)

In [ ]:
fpu.complete_processing(gossipcop_real, 1, "gossipcop_real_posts_3.csv", "gossipcop_real_features_3.csv", 2000, 3000)

In [ ]:
fpu.complete_processing(gossipcop_real, 1, "gossipcop_real_posts_4.csv", "gossipcop_real_features_4.csv", 3000, 4000)

In [ ]:
fpu.complete_processing(gossipcop_real, 1, "gossipcop_real_posts_5.csv", "gossipcop_real_features_5.csv", 4000, 5000)